In [6]:
%matplotlib inline
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os
import time
import copy
import random

from IPython.display import clear_output
from skimage.io import imread
from skimage.transform import resize

import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
from torch.nn import Linear, GRU, Conv2d, Dropout, MaxPool2d, BatchNorm1d
from torch.nn.functional import relu, elu, relu6, sigmoid, tanh, softmax
from skimage import io
from torchvision.io import read_image
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision.nets import resnet50, ResNet50_Weights
# from torchvision.nets.quantization import resnet50, ResNet50_QuantizedWeights
import torch.nn.functional as F
from torch.utils.data.sampler import SubsetRandomSampler


ModuleNotFoundError: No module named 'torchvision.nets'

In [ ]:
use_cuda = torch.cuda.is_available()
print("Running GPU.") if use_cuda else print("No GPU available.")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  

Running GPU.


# Load dataloaders and net

In [ ]:
class SelectItem(nn.Module):
    def __init__(self, item_index):
        super(SelectItem, self).__init__()
        self._name = 'selectitem'
        self.item_index = item_index

    def forward(self, inputs):
        return inputs[self.item_index]

In [ ]:
class LeafDataset(Dataset):
    """Leaf dataset."""

    def __init__(self, csv_file, root_dir, transform=None, train=False, test=False):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.leafs_df = pd.read_csv(csv_file)
        self.data = self.leafs_df.values
        self.root_dir = root_dir
        self.transform = transform
        self.train = train
        self.test = test

        self.parse_data()

    def parse_data(self):
        self.ids = np.array(self.data[:, 0], dtype=int)
        if self.train:
            self.labels = pd.Categorical(pd.factorize(self.leafs_df.species)[0])
            self.labels = np.array(self.labels)
            self.label_dummies = pd.get_dummies(self.leafs_df.species)
            self.label_dummies = np.array(self.label_dummies)
            self.species = np.array(self.data[:, 1], dtype=str)
            self.margins = np.array(self.data[:, 2:66], dtype=float)
            self.shapes = np.array(self.data[:, 66:130], dtype=float)
            self.textures = np.array(self.data[:, 130:], dtype=float)
        if self.test:
            self.labels = np.empty((len(self.data)), dtype=int)
            self.label_dummies = np.empty((len(self.data), NUM_CLASSES), dtype=int)
            self.species = np.empty((len(self.data)), dtype=str)
            self.margins = np.array(self.data[:, 1:65], dtype=float)
            self.shapes = np.array(self.data[:, 65:129], dtype=float)
            self.textures = np.array(self.data[:, 129:], dtype=float)


    def __len__(self):
        return len(self.leafs_df)

    def __getitem__(self, idx):
        # get the image
        img_name = str(self.ids[idx]) + '.jpg'
        img_path = os.path.join(self.root_dir, img_name)
        image = io.imread(img_path)

        # no matter what happens, we need to padd all the images to the same dimensions, so that we can resize them without distorting them
        image = data_utils.pad2square(image)  # Make the image square
        image = resize(image, output_shape=(128, 128), mode='reflect', anti_aliasing=True)  # resizes the image

        # augment the image if chosen to
        if self.transform:
            image = self.transform(image)

        # get the other data
        id_ = self.ids[idx]
        margin = self.margins[idx]
        shape = self.shapes[idx]
        texture = self.textures[idx]
        label = self.labels[idx]
        label_dummy = self.label_dummies[idx]
        specie = self.species[idx]
            
        return image, margin, shape, texture, label, label_dummy, specie, id_

In [ ]:
test_csv = 'test.csv'
root_dir = 'images/'
test_transform = transforms.Compose([transforms.ToTensor()])
batch_size = 32
testset = LeafDataset(test_csv, root_dir, transform=test_transform, train=False, test=True)
test_loader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=0)

In [ ]:
height, width, channels = 128,128,1

# Keep track of features to output layer
conv_feature_size = 768
vector_input_size = 128
vector_feature_size = 128
rnn_input_size = 64 # must be the same as the x_shape channels
rnn_feature_size = 128
features_cat_size = 1024

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        # prepare pretrained net
        resnet_weights = ResNet50_Weights.DEFAULT
        self.resnet = resnet50(weights=resnet_weights)
        self.resnet.eval()
        # Freeze the weights of the pre-trained net
        for param in self.resnet.parameters():
            param.requiresGrad = False
        num_ftrs = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_ftrs, conv_feature_size)
        # Step 2: Initialize the inference transforms
        self.preprocess = resnet_weights.transforms()

        ## margin, texture
        self.fc2 = nn.Sequential(
            nn.Linear(in_features=vector_input_size,
                    out_features=vector_feature_size,
                    bias=False),
            # nn.BatchNorm1d(128),
            nn.Dropout(0.5)
        )


        # shape
        self.recurrent = nn.Sequential(
            nn.GRU(input_size=rnn_input_size,     # The number of expected features in the input x
                    hidden_size=rnn_feature_size, # The number of features in the hidden state h
                    num_layers=2),                # Number of recurrent layers
            SelectItem(0)
        )

        # classification
        self.l_out = nn.Sequential(
            nn.Linear(in_features=features_cat_size,
                        out_features=NUM_CLASSES,
                        bias=True)
        )
        
        
    def forward(self, X):
        X_img, x_margin, x_shape, x_texture = X
        X_img = self.preprocess(X_img)
        features = []


        features_img = self.resnet(X_img)

        features.append(features_img)
        
        # Use concatenated leaf features for FFNN ##
        # print("\nFeed Forward...")
        x = torch.cat((x_margin, x_texture), dim=1)  # if you want to use features as feature vectors
        x = self.fc2(x)
        features_vector = x
        features.append(features_vector)
        
        ## Use concatenated leaf features for RNN ##
        # print("\nRecurrent...")
        features_rnn = self.recurrent(x_shape)
        features.append(features_rnn)
        
        ## Output layer where all features are in use ##
        # print("\Features...")
        features_final = torch.cat(features, dim=1)
        
        # print("\nOutput...")
        out = self.l_out(features_final)
       
        return out, F.softmax(out, dim=1)

net = Net()
net = net.float()
if use_cuda:
    net.to(device)
print(net)

Net(
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): 

In [ ]:
net.load_state_dict(torch.load("/zhome/68/a/154632/Documents/dl/LeafClassification/my_net.pth"))

<All keys matched successfully>

# Submission to Kaggle

First we have to make test set predictions, then we have to place the output in the submission file and then upload to Kaggle to get our score! You can upload up to 5 submissions per day.

In [ ]:
# show intermediate results
# Compute the val accuracy
ids_test, preds_test = [], []
net.eval()  # testing mode
for (num, batch) in enumerate(test_loader):
    # extract subfields
    image, margins, shapes, textures, _, _, _, ids = batch
    image = np.repeat(image, 3, axis=1)
    
    # convert to float and move to cuda
    image = image.to(device).float()
    margins = margins.to(device).float()
    shapes = shapes.to(device).float()
    textures = textures.to(device).float()

    # split input and label
    inputs = image, margins, shapes, textures

    y_out, _ = net(inputs)
    y_out = y_out.detach().cpu().numpy()

    ids = ids.detach().cpu().numpy()
    print(ids, np.shape(ids))
    ids_test.append(ids)
    if num!=len(y_out):
        # in case of the last batch, num will be less than batch_size
        y_out = y_out[:num]
    print(y_out, np.shape(y_out))
    preds_test.append(y_out)
preds_test = np.concatenate(preds_test, axis=0)

assert len(ids_test) == len(preds_test)

OutOfMemoryError: CUDA out of memory. Tried to allocate 26.00 MiB (GPU 0; 39.42 GiB total capacity; 20.66 GiB already allocated; 21.06 MiB free; 21.16 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

## Make submission file

In [ ]:
preds_df = pd.DataFrame(preds_test, columns=data.le.classes_)
ids_test_df = pd.DataFrame(ids_test, columns=["id"])
submission = pd.concat([ids_test_df, preds_df], axis=1)
submission.to_csv(drive_path + 'submission.csv', index=False)

# below prints the submission, can be removed and replaced with code block below
submission.head(5)

## Upload submission

1. Go to [`https://www.kaggle.com/c/leaf-classification/submit`](https://www.kaggle.com/c/leaf-classification/submit)
3. Click or drop your submission here (writing a description is good practice)
4. Submit and look at where you are on the leaderboard.

Success! 